In [ ]:
SEED = 0

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [ ]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [ ]:
!kaggle datasets download -d mlg-ulb/creditcardfraud

creditcardfraud.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip \*.zip  && rm *.zip

Archive:  creditcardfraud.zip
replace creditcard.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
rm: cannot remove '*.zip': No such file or directory


In [ ]:
!ls

creditcard.csv	kaggle.json


In [ ]:
#Loading data
import pandas as pd
import numpy as np

data = pd.read_csv("/content/gdrive/My Drive/Kaggle/creditcard.csv", header=0)
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
#Handle Duplicated values
print("Duplicates", data.duplicated().sum())
data = data.drop_duplicates()
print("Rows {}, Columns {}".format(data.shape[0], data.shape[1]))

Duplicates 1081
Rows 283726, Columns 31


In [ ]:
data['Class'] = data['Class'].replace({'0': 0, '1': 1})
data['Class'].unique()
data['Class'].value_counts()

<ipython-input-7-dc5853bfac52>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Class'] = data['Class'].replace({'0': 0, '1': 1})


0    283253
1       473
Name: Class, dtype: int64

In [ ]:
labels = data['Class'] 
data = data[data.columns[:-1]]
labels.value_counts()

0    283253
1       473
Name: Class, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( data, labels, test_size=0.3, random_state=SEED )

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10, random_state=SEED).fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [ ]:
#Cost sensitive KNN

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer, accuracy_score, recall_score
from sklearn.model_selection import GridSearchCV
import numpy as np

cost_matrix = np.array([[0, 1],
 [10, 0]])

# Define the KNN classifier
knn = KNeighborsClassifier(algorithm='brute')

# Define a custom distance metric that computes the cost of misclassifying an instance based on the cost matrix
def cost_sensitive_recall(y_true, y_pred):
    cost = 0
    print("Ytrue", y_true)
    for i in range(len(y_true)):
        if y_true[i] != y_pred[i]:
            cost += cost_matrix[y_true[i], y_pred[i]]
    recall = recall_score(y_true, y_pred)
    return recall - cost/len(y_true)


# Define the parameter grid for grid search
param_grid = {'n_neighbors': [x for x in range(1, 15)], 'weights': ['uniform', 'distance']}

# Define the scorer based on the custom cost-sensitive metric
scorer = make_scorer(lambda y_true, y_pred: accuracy_score(y_true, y_pred, sample_weight=[cost_sensitive_recall[y_true[i], y_pred[i]] for i in range(len(y_true))]), greater_is_better=True)


In [ ]:
grid_search = GridSearchCV(knn, param_grid, scoring=scorer)

# Set the custom distance metric as the "metric" parameter of the KNN classifier
grid_search.set_params(estimator__metric=cost_sensitive_recall)

# Train the grid search object on the training set
grid_search.fit(X_train_pca, y_train)

Ytrue [-1.50225274e+04  2.67526064e+01 -2.12723890e+00  6.15987080e-01
 -6.22688098e-01  1.48180964e+00  4.23139380e-01  5.21296300e-01
  1.74675124e-01  8.40102263e-01]
Ytrue [ 2.72754663e+04 -8.95741138e+01  1.79042228e+00 -5.02652943e-01
 -2.06716493e-01  3.50145769e-01  5.29391375e-01 -4.20493649e-01
 -7.27107215e-01  7.13659587e-02]
Ytrue [ 7.03046635e+03 -8.74632642e+01  3.47713163e+00 -1.74083251e+00
 -2.49365519e+00  1.63273761e+00  7.27582901e-01 -1.09895994e+00
 -1.07165956e+00  3.37780885e-01]
Ytrue [-2.02515336e+04 -8.60727837e+01  2.72825273e+00 -1.99731363e+00
  4.66212950e-01  2.64634144e-01 -8.80223455e-01 -4.04556205e-01
  6.29698748e-01  4.73677738e-01]
Ytrue [ 1.35264713e+04  1.18457920e+00  2.89953336e+00 -1.04486425e+00
  1.23316783e-01 -4.80508998e-02 -1.35249726e+00  1.04159901e+00
  3.12181821e+00 -5.02160477e-01]
Ytrue [-1.50225274e+04  2.67526064e+01 -2.12723890e+00  6.15987080e-01
 -6.22688098e-01  1.48180964e+00  4.23139380e-01  5.21296300e-01
  1.74675124e-

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_scorer.py", line 276, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/neighbors/_classification.py", line 234, in predict
    neigh_ind = self.kneighbors(X, return_distance=False)
  File "/usr/loca

Ytrue [ 1.35264713e+04  1.18457920e+00  2.89953336e+00 -1.04486425e+00
  1.23316783e-01 -4.80508998e-02 -1.35249726e+00  1.04159901e+00
  3.12181821e+00 -5.02160477e-01]
Ytrue [-1.50225274e+04  2.67526064e+01 -2.12723890e+00  6.15987080e-01
 -6.22688098e-01  1.48180964e+00  4.23139380e-01  5.21296300e-01
  1.74675124e-01  8.40102263e-01]
Ytrue [ 2.72754663e+04 -8.95741138e+01  1.79042228e+00 -5.02652943e-01
 -2.06716493e-01  3.50145769e-01  5.29391375e-01 -4.20493649e-01
 -7.27107215e-01  7.13659587e-02]
Ytrue [ 7.03046635e+03 -8.74632642e+01  3.47713163e+00 -1.74083251e+00
 -2.49365519e+00  1.63273761e+00  7.27582901e-01 -1.09895994e+00
 -1.07165956e+00  3.37780885e-01]
Ytrue [-2.02515336e+04 -8.60727837e+01  2.72825273e+00 -1.99731363e+00
  4.66212950e-01  2.64634144e-01 -8.80223455e-01 -4.04556205e-01
  6.29698748e-01  4.73677738e-01]
Ytrue [ 1.35264713e+04  1.18457920e+00  2.89953336e+00 -1.04486425e+00
  1.23316783e-01 -4.80508998e-02 -1.35249726e+00  1.04159901e+00
  3.12181821e+

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_scorer.py", line 276, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/neighbors/_classification.py", line 237, in predict
    neigh_dist, neigh_ind = self.kneighbors(X)
  File "/usr/local/lib/pytho

Ytrue [ 7.03046635e+03 -8.74632642e+01  3.47713163e+00 -1.74083251e+00
 -2.49365519e+00  1.63273761e+00  7.27582901e-01 -1.09895994e+00
 -1.07165956e+00  3.37780885e-01]
Ytrue [-2.02515336e+04 -8.60727837e+01  2.72825273e+00 -1.99731363e+00
  4.66212950e-01  2.64634144e-01 -8.80223455e-01 -4.04556205e-01
  6.29698748e-01  4.73677738e-01]
Ytrue [ 1.35264713e+04  1.18457920e+00  2.89953336e+00 -1.04486425e+00
  1.23316783e-01 -4.80508998e-02 -1.35249726e+00  1.04159901e+00
  3.12181821e+00 -5.02160477e-01]
Ytrue [-1.50225274e+04  2.67526064e+01 -2.12723890e+00  6.15987080e-01
 -6.22688098e-01  1.48180964e+00  4.23139380e-01  5.21296300e-01
  1.74675124e-01  8.40102263e-01]
Ytrue [ 2.72754663e+04 -8.95741138e+01  1.79042228e+00 -5.02652943e-01
 -2.06716493e-01  3.50145769e-01  5.29391375e-01 -4.20493649e-01
 -7.27107215e-01  7.13659587e-02]
Ytrue [ 7.03046635e+03 -8.74632642e+01  3.47713163e+00 -1.74083251e+00
 -2.49365519e+00  1.63273761e+00  7.27582901e-01 -1.09895994e+00
 -1.07165956e+

...

Ytrue [ 7.03046635e+03 -8.74632642e+01  3.47713163e+00 -1.74083251e+00
 -2.49365519e+00  1.63273761e+00  7.27582901e-01 -1.09895994e+00
 -1.07165956e+00  3.37780885e-01]
Ytrue [-2.02515336e+04 -8.60727837e+01  2.72825273e+00 -1.99731363e+00
  4.66212950e-01  2.64634144e-01 -8.80223455e-01 -4.04556205e-01
  6.29698748e-01  4.73677738e-01]
Ytrue [ 1.35264713e+04  1.18457920e+00  2.89953336e+00 -1.04486425e+00
  1.23316783e-01 -4.80508998e-02 -1.35249726e+00  1.04159901e+00
  3.12181821e+00 -5.02160477e-01]
Ytrue [-1.50225274e+04  2.67526064e+01 -2.12723890e+00  6.15987080e-01
 -6.22688098e-01  1.48180964e+00  4.23139380e-01  5.21296300e-01
  1.74675124e-01  8.40102263e-01]
Ytrue [ 2.72754663e+04 -8.95741138e+01  1.79042228e+00 -5.02652943e-01
 -2.06716493e-01  3.50145769e-01  5.29391375e-01 -4.20493649e-01
 -7.27107215e-01  7.13659587e-02]
Ytrue [ 7.03046635e+03 -8.74632642e+01  3.47713163e+00 -1.74083251e+00
 -2.49365519e+00  1.63273761e+00  7.27582901e-01 -1.09895994e+00
 -1.07165956e+

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_scorer.py", line 276, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/neighbors/_classification.py", line 234, in predict
    neigh_ind = self.kneighbors(X, return_distance=False)
  File "/usr/loca

Ytrue [ 2.72754663e+04 -8.95741138e+01  1.79042228e+00 -5.02652943e-01
 -2.06716493e-01  3.50145769e-01  5.29391375e-01 -4.20493649e-01
 -7.27107215e-01  7.13659587e-02]
Ytrue [ 7.03046635e+03 -8.74632642e+01  3.47713163e+00 -1.74083251e+00
 -2.49365519e+00  1.63273761e+00  7.27582901e-01 -1.09895994e+00
 -1.07165956e+00  3.37780885e-01]
Ytrue [-2.02515336e+04 -8.60727837e+01  2.72825273e+00 -1.99731363e+00
  4.66212950e-01  2.64634144e-01 -8.80223455e-01 -4.04556205e-01
  6.29698748e-01  4.73677738e-01]
Ytrue [ 1.35264713e+04  1.18457920e+00  2.89953336e+00 -1.04486425e+00
  1.23316783e-01 -4.80508998e-02 -1.35249726e+00  1.04159901e+00
  3.12181821e+00 -5.02160477e-01]
Ytrue [-1.50225274e+04  2.67526064e+01 -2.12723890e+00  6.15987080e-01
 -6.22688098e-01  1.48180964e+00  4.23139380e-01  5.21296300e-01
  1.74675124e-01  8.40102263e-01]
Ytrue [ 2.72754663e+04 -8.95741138e+01  1.79042228e+00 -5.02652943e-01
 -2.06716493e-01  3.50145769e-01  5.29391375e-01 -4.20493649e-01
 -7.27107215e-

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_scorer.py", line 276, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/neighbors/_classification.py", line 234, in predict
    neigh_ind = self.kneighbors(X, return_distance=False)
  File "/usr/loca

GridSearchCV(estimator=KNeighborsClassifier(algorithm='brute',
                                            metric=<function cost_sensitive_recall at 0x7f3ccf172040>),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14],
                         'weights': ['uniform', 'distance']},
             scoring=make_scorer(<lambda>))

In [ ]:
# Evaluate the performance of the best estimator on the testing set
y_pred = grid_search.predict(X_test_pca)
acc = accuracy_score(y_test, y_pred, sample_weight=[cost_matrix[y_test[i], y_pred[i]] for i in range(len(y_test))])

print("Best parameters:", grid_search.best_params_)
print("Best accuracy:", grid_search.best_score_)
print("Accuracy on testing set:", acc)

Ytrue [-6.74335335e+04 -8.31873756e+01 -1.67861005e+00  2.39886544e-01
  1.80366284e-01  1.71533106e-01  4.59680416e-01  1.00564000e+00
 -2.63130903e-01  1.03405667e+00]


IndexError: ignored

In [ ]:
weight_range = np.logspace(0, 3, 10)

knn = KNeighborsClassifier()

# Define a custom distance metric that computes the cost of misclassifying an instance based on the cost matrix
def cost_sensitive_distance(x1, x2):
    cost_matrix = np.array([[0, 1], [10, 0]]) # Define the cost matrix
    cost = 0
    for i in range(len(x1)):
        x1[i] = int(x1[i])
        x2[i] = int(x2[i])
        if x1[i] != x2[i]:
            cost += cost_matrix[x1[i], x2[i]]
    return np.sqrt(np.sum(np.square(x1 - x2))) + cost

# Define the parameter grid for grid search
param_grid = {'n_neighbors': [x for x in range(1, 15)], 'weights': ['uniform', 'distance']}

# Define the scorer based on the custom cost-sensitive metric
scorer = make_scorer(accuracy_score, greater_is_better=True)

grid_search = GridSearchCV(knn, param_grid, scoring=scorer)

# Set the custom distance metric as the "metric" parameter of the KNN classifier
grid_search.set_params(estimator__metric=cost_sensitive_distance)

# Train the grid search object on the training set
grid_search.fit(X_train_pca, y_train)






ValueError: ignored

In [ ]:
#svm

import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer, recall_score, accuracy_score
import numpy as np

'''# fit the model and get the separating hyperplane using weighted classes
wclf = svm.SVC(kernel="linear", class_weight={1: 10})
wclf.fit(X_train, y_train)'''

# plot the samples
wclf = svm.SVC()
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=SEED)
scorer = make_scorer(recall_score, pos_label=1)

weight_range = np.logspace(0, 3, 10)
weight_range = np.append(weight_range, 577)

param_grid = {'C': [0.1, 1, 10, 100], 'class_weight': [{0:1, 1:w} for w in weight_range]}
print(param_grid)

{'C': [0.1, 1, 10, 100], 'class_weight': [{0: 1, 1: 1.0}, {0: 1, 1: 2.154434690031884}, {0: 1, 1: 4.641588833612778}, {0: 1, 1: 10.0}, {0: 1, 1: 21.544346900318832}, {0: 1, 1: 46.41588833612777}, {0: 1, 1: 100.0}, {0: 1, 1: 215.44346900318823}, {0: 1, 1: 464.15888336127773}, {0: 1, 1: 1000.0}, {0: 1, 1: 577.0}]}


In [ ]:
grid_search = GridSearchCV(wclf, param_grid=param_grid, cv=cv, scoring=scorer, verbose = 3)
grid_result = grid_search.fit(X_train_pca, y_train)

Fitting 10 folds for each of 44 candidates, totalling 440 fits
[CV 1/10] END C=0.1, class_weight={0: 1, 1: 1.0};, score=0.000 total time=   9.1s
[CV 2/10] END C=0.1, class_weight={0: 1, 1: 1.0};, score=0.000 total time=  15.3s
[CV 3/10] END C=0.1, class_weight={0: 1, 1: 1.0};, score=0.000 total time=   5.7s
[CV 4/10] END C=0.1, class_weight={0: 1, 1: 1.0};, score=0.000 total time=   7.7s
[CV 5/10] END C=0.1, class_weight={0: 1, 1: 1.0};, score=0.000 total time=   5.7s
[CV 6/10] END C=0.1, class_weight={0: 1, 1: 1.0};, score=0.000 total time=   7.7s
[CV 7/10] END C=0.1, class_weight={0: 1, 1: 1.0};, score=0.000 total time=   5.6s
[CV 8/10] END C=0.1, class_weight={0: 1, 1: 1.0};, score=0.000 total time=   7.6s
[CV 9/10] END C=0.1, class_weight={0: 1, 1: 1.0};, score=0.000 total time=   5.6s
[CV 10/10] END C=0.1, class_weight={0: 1, 1: 1.0};, score=0.000 total time=   7.4s
[CV 1/10] END C=0.1, class_weight={0: 1, 1: 2.154434690031884};, score=0.000 total time=  14.1s
[CV 2/10] END C=0.1,

In [ ]:
dec_val = grid_result.best_estimator_ 
val_y_pred = dec_val.predict(X_test_pca) 
print(accuracy_score(val_y_pred,y_test))
confusion_matrix(y_test, val_y_pred)

from sklearn.metrics import ConfusionMatrixDisplay
print("\n", ConfusionMatrixDisplay.from_estimator(grid_result.best_estimator_ , X_test_pca, y_test, display_labels=labels.unique()))

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
import numpy as np

cost_matrix = np.array([[0, 1],
                        [10, 0]])

# Define the SVM classifier
svm = SVC(kernel='linear')

# Define a custom scoring metric that computes the cost of misclassification based on the cost matrix
def cost_sensitive_score(y_true, y_pred):
    y_true = y_true.astype(int)
    y_pred = y_pred.astype(int)
    cost = 0
    for i in range(len(y_true)):
        if y_true[i] != y_pred[i]:
            cost += cost_matrix[y_true[i], y_pred[i]]
    return -cost  # Return the negative cost to maximize the score

# Define the parameter grid for grid search
param_grid = {'C': [0.1, 1, 10, 100], 'class_weight': [None, 'balanced']}

# Define the scorer based on the custom cost-sensitive metric
scorer = make_scorer(cost_sensitive_score, greater_is_better=True)

# Define the grid search object
grid_search = GridSearchCV(svm, param_grid=param_grid, cv=5, scoring=scorer)



KeyboardInterrupt: ignored

In [ ]:
# Train the grid search object on the training set
grid_search.fit(X_train, y_train)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", -grid_search.best_score_)


/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pandas/core/indexes/base.py", line 3629, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 136, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 163, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 2131, in pandas._libs.hashtable.Int64HashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 2140, in pandas._libs.hashtable.Int64HashTable.get_item
KeyError: 0

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
  